In [7]:
import sqlite3
import pandas as pd

class feedzaiChallenge():
    def __init__(self,database_name):
        self.database_name = database_name
        self.conn = sqlite3.connect(f'{self.database_name}.db')

    def read_csv_file(self, csv_path:str, read_params:dict=None):
        try:
            self.df_csv = pd.read_csv(csv_path) if read_params == None else pd.read_csv(csv_path,**read_params)
        except Exception as ex:
            raise ex
    
    def load_data(self, table_name:str):
        try:
            self.df_csv.to_sql(table_name, self.conn, if_exists='replace', index=False)
        except Exception as ex:
            raise ex
    
    def query_acumulated_actual_costs(self,output_path:str):
        try:
            query = f"""
            WITH daily_accumulated_cost AS (
                SELECT
                    project_id,
                    date,
                    SUM(worked) * 100 AS accumulated_cost
                FROM
                    work_hours
                GROUP BY
                    project_id, date
                ORDER BY
                    project_id, date
            )
            SELECT
                project_id,
                date,
                SUM(accumulated_cost) OVER (PARTITION BY project_id ORDER BY date) AS total_accumulated_cost
            FROM
                daily_accumulated_cost;
            """
            
            result = pd.read_sql_query(query, self.conn)
            result.to_csv(output_path, index=False)
        except Exception as ex:
            raise ex
        
    def close_connection(self):
        self.conn.close()


In [8]:
feedzai = feedzaiChallenge('feedzai_database')

In [9]:
feedzai.read_csv_file('time_off.csv')
feedzai.load_data('time_off')

In [10]:
feedzai.read_csv_file('work_hours.csv')
feedzai.load_data('work_hours')

In [11]:
feedzai.query_acumulated_actual_costs('acumulated_actual_costs.csv')


In [12]:
feedzai.close_connection()